In [8]:
!pip install credentials

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf

In [1]:
import tweepy
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from time import sleep
from credentials import *

# load ascii text and covert to lowercase
filename = "C:/Users/Sushant/Documents/Python Scripts/text_corpus.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 140
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="C:/Users/Sushant/Documents/Python Scripts/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)


Using TensorFlow backend.


Total Characters:  1115394
Total Vocab:  39
Total Patterns:  1115254


C:\Users\Sushant\Anaconda3\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
1115254/1115254 [==============================] - 251s - loss: 3.2282   


OSError: Unable to open file (Unable to open file: name = 'c:/users/sushant/documents/python scripts/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5', errno = 22, error message = 'invalid argument', flags = 0, o_flags = 0)

In [2]:
import sys
# load the network weights
filename = "C:/Users/Sushant/Documents/Python Scripts/weights-improvement-00-3.2282.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed: ")
print (''.join([int_to_char[value] for value in pattern]))
# generate characters
for i in range(140):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
fTweet = ''.join([int_to_char[value] for value in pattern])
print ("\nDone.")


Seed: 
eople will remain uncertain whilst
'twixt you there's difference; but the fall of either
makes the survivor heir of all.

aufidius:
i know i
                                                                                                    
Done.


In [28]:
import tweepy
import numpy as np
import io
from time import sleep
from credentials import *

consumer_key = '02bAf8hfsxomd5XEZM77bKH0H'
consumer_secret = 'jrzAakIvFmvrMVOC0wNuPn8jxbgupTx9dvR1NcLhGTcFBbBzLU'
access_token = '160171403-QeTmzG8BuEtZMsWcN3EEL4cdmmHUdyc24XOFdVGZ'
access_token_secret = 'R0lvKBTKASiNPjaQwFNEdneHINlVcB4k0tjWztMcwddyw'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(consumer_secret, access_token_secret)
api = tweepy.API(auth)

# Set up OAuth and integrate with API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Write a tweet to push to our Twitter account
for i in range(20):
    try:
        tweet = "tweet" + str(i+1) + ": " + str(fTweet)
        tweet = tweet[:140]
        api.update_status(tweet)
        
    except tweepy.TweepError as e:
        print(e.reason)
    sleep(360)